In [25]:
#Esto me sirve para el correcto funcionamiento de las funciones importadas en este notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sistema de recomendación basado en conocimiento

## Contexto

Siguiendo en la línea de lo que hicimos [en el notebook anterior](1_IMDB_TopClon.ipynb), ahora voy a dar un paso más con el propósito de construir lo que se conoce como un **sistema de recomendación basado en conocimiento**. Estos sistemas sugieren productos o servicios a los usuarios **utilizando información específica sobre el producto y las preferencias del usuario**. A diferencia de otros métodos, se basan en reglas y hechos explícitos, como las características del producto y las necesidades del usuario, para hacer recomendaciones precisas y personalizadas.

Para nuestro caso este sistema de recomendación va a ser una simple función en Python que seguirá los siguientes pasos:

1. Pregunta al usuario por el género de la película que busca
2. Pregunta al usuario por la duración de película que desea
3. Pregunta al usuario por un período de tiempo en años que definan la fecha de estreno de la película
4. Usando toda la información anterior le recomienda al usuario una serie de películas que tienen un valor alto en la métrica definida anteriormente (la fórmula de peso de IMDB) y que además cumplan con todas las condiciones especificadas

## Preprocesando los datos

Como siempre, empiezo por el cargado de los datos a usar:

In [26]:
#Importo las librerías a usar
from utils.paths import crear_funcion_directorio
import pandas as pd
from ast import literal_eval

#Creo un acceso directo a la carpeta de data
data_dir = crear_funcion_directorio("data")

#Cargo el dataset
movies_metadata_df = pd.read_csv(data_dir("raw", "movies_metadata.csv"), low_memory= False)

Le doy ahora un vistazo a las columnas pero solo viendo su nombre:

In [27]:
# Imprimo las columnas del dataset
movies_metadata_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Podemos ver que de aquí hay muchas columnas que no nos son útiles para lo que queremos hacer, por lo tanto un paso lógico sería reducir las columnas en el dataframe para poder construir el sistema de recomendación tranquilamente:

In [28]:
movies_metadata_df_filtered = movies_metadata_df[["title", "genres", "release_date", "runtime", "vote_average", "vote_count"]].copy()

movies_metadata_df_filtered

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0
...,...,...,...,...,...,...
45461,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,90.0,4.0,1.0
45462,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",2011-11-17,360.0,9.0,3.0
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",2003-08-01,90.0,3.8,6.0
45464,Satan Triumphant,[],1917-10-21,87.0,0.0,0.0


Podemos ver que hay varias cosas que limpiar y modificar en este dataframe, un claro caso son las columnas **'genres'** y **'release_date'** que no están en el formato que deseamos. Para lograr esta modificación hay que notar varias cosas:

* **'genres'** es una lista de diccionarios que como clave tienen un id del género y como valor el nombre del mismo, para efectos de lo que se desea construir solo nos servirá tener el nombre, por lo tanto esta columna debería terminar siendo una lista que solo tenga los nombres de los géneros
* **'release_date'** viene dada por una fecha completa pero a nosotros solo nos interesa el año, por lo tanto debemos extraerlo y convertirlo a un tipo entero

Explicados estos dos casos, procedo a generar las modificaciones:

In [29]:
# Convierto 'release_date' a datetime
movies_metadata_df_filtered["release_date"] = pd.to_datetime(movies_metadata_df_filtered["release_date"], errors='coerce')

# Extraigo el año de 'release_date' y lo hago de tipo entero
movies_metadata_df_filtered["release_year"] = movies_metadata_df_filtered["release_date"].dt.year.astype("Int64")

# Elimino 'release_date'
movies_metadata_df_filtered.drop("release_date", axis=1, inplace=True)

# Convierto 'genres' a lista de diccionarios
movies_metadata_df_filtered["genres"] = movies_metadata_df_filtered["genres"].apply(literal_eval)

# Extraigo los nombres de los géneros
movies_metadata_df_filtered["genres"] = movies_metadata_df_filtered["genres"].apply(lambda x: [i["name"] for i in x])

# Lo imprimo en pantalla
movies_metadata_df_filtered

,title,genres,runtime,vote_average,vote_count,release_year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995
...,...,...,...,...,...,...
45461,Subdue,"[Drama, Family]",90.0,4.0,1.0,<NA>
45462,Century of Birthing,[Drama],360.0,9.0,3.0,2011
45463,Betrayal,"[Action, Drama, Thriller]",90.0,3.8,6.0,2003
45464,Satan Triumphant,[],87.0,0.0,0.0,1917


## Construyendo la función y poniéndola a prueba

Ahora que tenemos todas las modificaciones necesarios estamos aptos para construir la función que contendrá a nuestro **sistema de recomendación**. Esta función seguirá el flujo descrito anteriormente pero también hay que notar algo importante, y es que los valores de **C** (el promedio del rating de la película) y de **m** (la cantidad de votos mínimos para una película que toleraremos) ya no serán los mismos que antes, ahora estos se calculan según el filtro que indique el usuario. 

Sin más que decir, procedo a escribir la función:

In [30]:
def recommend_movie(df_gen:pd.DataFrame, genero:str, dur_min:int, 
                    dur_max:int, anio_min:int, anio_max:int, percentile:float=0.85):
    """
    Esta función recibe especifcaciones indicadas por el usuario y recomienda
    una película en base a la cantidad de votos y a la calificación promedio de las que 
    cumplan con las características.
    """

    # Filtro las películas que cumplen con las características
    df_gen = df_gen[(df_gen["genres"].apply(lambda x: genero in x)) & 
                    (df_gen["runtime"] >= dur_min) & 
                    (df_gen["runtime"] <= dur_max) & 
                    (df_gen["release_year"] >= anio_min) & 
                    (df_gen["release_year"] <= anio_max)].copy()
    
    # Calculo el número mínimo de votos requeridos para estar en el percentil deseado
    m = df_gen["vote_count"].quantile(percentile)
    
    # Calculo el promedio de votos
    C = df_gen["vote_average"].mean()
    
    # Filtro las películas que cumplen con el mínimo de votos
    movies = df_gen[df_gen["vote_count"] >= m].copy()
    
    # Calculo la calificación ponderada
    movies["score"] = (movies["vote_count"]/(movies["vote_count"]+m) * movies["vote_average"]) + (m/(m+movies["vote_count"]) * C)
    
    # Ordeno las películas por score y devuelvo las 10 primeras
    return movies.sort_values("score", ascending=False).head(10)

Ahora queda probar nuestro **sistema de recomendación** con diferentes combinaciones de características:

In [31]:
recommend_movie(movies_metadata_df_filtered, genero= "Science Fiction", dur_min= 90, dur_max= 120, anio_min= 2000, anio_max= 2020)

,title,genres,runtime,vote_average,vote_count,release_year,score
7208,Eternal Sunshine of the Spotless Mind,"[Science Fiction, Drama, Romance]",108.0,7.9,3758.0,2004,7.487984
23465,Edge of Tomorrow,"[Action, Science Fiction]",113.0,7.6,4979.0,2014,7.313846
24482,Ex Machina,"[Drama, Science Fiction]",108.0,7.6,4862.0,2015,7.307837
26553,Mad Max: Fury Road,"[Action, Adventure, Science Fiction, Thriller]",120.0,7.3,9629.0,2015,7.163143
21592,Gravity,"[Science Fiction, Thriller, Drama]",91.0,7.3,5879.0,2013,7.085250
40598,Arrival,"[Thriller, Drama, Science Fiction, Mystery]",113.0,7.2,5729.0,2016,6.991270
13966,District 9,[Science Fiction],112.0,7.3,3451.0,2009,6.959933
13622,Moon,"[Science Fiction, Drama]",97.0,7.6,1831.0,2009,6.959300
26568,Doctor Strange,"[Action, Adventure, Fantasy, Science Fiction]",115.0,7.1,5880.0,2016,6.906824
11353,Children of Men,"[Drama, Action, Thriller, Science Fiction]",109.0,7.4,2120.0,2006,6.874898


In [32]:
recommend_movie(movies_metadata_df_filtered, genero= "Action", dur_min= 50, dur_max= 300, anio_min= 1980, anio_max= 2020)

,title,genres,runtime,vote_average,vote_count,release_year,score
12481,The Dark Knight,"[Drama, Action, Crime, Thriller]",152.0,8.3,12269.0,2008,8.215697
1154,The Empire Strikes Back,"[Adventure, Action, Science Fiction]",124.0,8.2,5998.0,1980,8.039154
15480,Inception,"[Action, Thriller, Science Fiction, Mystery, A...",148.0,8.1,14075.0,2010,8.031666
7000,The Lord of the Rings: The Return of the King,"[Adventure, Fantasy, Action]",201.0,8.1,8226.0,2003,7.985299
4863,The Lord of the Rings: The Fellowship of the Ring,"[Adventure, Fantasy, Action]",178.0,8.0,8892.0,2001,7.897768
5814,The Lord of the Rings: The Two Towers,"[Adventure, Fantasy, Action]",179.0,8.0,7641.0,2002,7.881851
23753,Guardians of the Galaxy,"[Action, Science Fiction, Adventure]",121.0,7.9,10014.0,2014,7.812574
2458,The Matrix,"[Action, Science Fiction]",136.0,7.9,9079.0,1999,7.803945
13605,Inglourious Basterds,"[Drama, Action, Thriller, War]",153.0,7.9,6598.0,2009,7.769862
3456,Gladiator,"[Action, Drama, Adventure]",155.0,7.9,5566.0,2000,7.747328


In [33]:
recommend_movie(movies_metadata_df_filtered, genero= "Comedy", dur_min= 50, dur_max= 300, anio_min= 2000, anio_max= 2020)

,title,genres,runtime,vote_average,vote_count,release_year,score
18465,The Intouchables,"[Drama, Comedy]",112.0,8.2,5410.0,2011,8.108798
22841,The Grand Budapest Hotel,"[Comedy, Drama]",99.0,8.0,4644.0,2014,7.902847
22131,The Wolf of Wall Street,"[Crime, Drama, Comedy]",180.0,7.9,6768.0,2013,7.835380
30315,Inside Out,"[Drama, Comedy, Animation, Family]",94.0,7.9,6737.0,2015,7.835091
40882,La La Land,"[Comedy, Drama, Music, Romance]",128.0,7.9,4745.0,2016,7.808970
13724,Up,"[Animation, Comedy, Family, Adventure]",96.0,7.8,7048.0,2009,7.740701
24455,Big Hero 6,"[Adventure, Family, Animation, Action, Comedy]",102.0,7.8,6289.0,2014,7.733770
4843,Amélie,"[Comedy, Romance]",122.0,7.8,3403.0,2001,7.680794
38798,Captain Fantastic,"[Adventure, Comedy, Drama, Romance]",119.0,7.9,1569.0,2016,7.646000
36253,Zootopia,"[Animation, Adventure, Family, Comedy]",108.0,7.7,4961.0,2016,7.620713


Podemos ver que se logró generar un **sistema de recomendación basado en conocimiento** que es bastante acertado, siendo que las recomendaciones que retorna cumplen perfectamente las características que indicamos, este sigue siendo un sistema aún básico pero más fino que lo que habíamos construido anteriormente.

Finalmente guardo **'movies_metadata_df_filtered'** en un archivo CSV para usarlo más adelante:

In [34]:
movies_metadata_df_filtered.to_csv(data_dir("processed", "movies_metadata_filtered.csv"), index=False)